In [4]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from circadian.models import Skeldon23
from circadian.lights import LightSchedule
from utils.helper_functions import H_thresholds

In [5]:
save_path = "data/chang14_reference_parameters/"
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [6]:
light_data_location = 'data/light_schedules/'

# With forced wake up

In [7]:
# Equilibrate the model
dt = 0.005 # hours
equilibration_days = 3
equilibration_loops = 10
equilibration_time = np.arange(0.0, 24 * equilibration_days, dt)
lights_on = 6.0
lights_off = 22.0
typical_indoor_lux = 1000.0
initial_sleep = 1.0 # asleep
forced_wakeup_threshold = 0.5 # lux
regular_schedule = LightSchedule.Regular(typical_indoor_lux, lights_on=lights_on,
                                         lights_off=lights_off)
equilibration_light = regular_schedule(equilibration_time)
# Equilibrate
parameters = {'forced_wakeup_threshold': forced_wakeup_threshold,
              'mu': 19.0,
              'chi': 11.0,
              'Delta': 6.0,
              'p': 0.6,
              'tauc': 24.2,
              'k': 0.55,
              }
equilibration_model = Skeldon23(params=parameters)
initial_condition = equilibration_model.equilibrate(equilibration_time, 
                                                    equilibration_light,
                                                    num_loops=equilibration_loops,)


In [8]:
time = np.load(f"{light_data_location}/time.npy")

chang14_ebook_first_light = np.load(f"{light_data_location}/chang14_ebook_first_light.npy")
chang14_ebook_second_light = np.load(f"{light_data_location}/chang14_ebook_second_light.npy")
chang14_high_ebook_first_light = np.load(f"{light_data_location}/chang14_high_ebook_first_light.npy")
chang14_high_ebook_second_light = np.load(f"{light_data_location}/chang14_high_ebook_second_light.npy")
realistic_ebook_first_light = np.load(f"{light_data_location}/realistic_ebook_first_light.npy")
realistic_ebook_second_light = np.load(f"{light_data_location}/realistic_ebook_second_light.npy")

parameters = {'forced_wakeup_threshold': forced_wakeup_threshold,
              'mu': 19.0,
              'chi': 11.0,
              'Delta': 6.0,
              'p': 0.6,
              'tauc': 24.2,
              'k': 0.55,
              }

# Chang14 Ebook first simulation
chang14_ebook_first_model = Skeldon23(params=parameters)
chang14_ebook_first_trajectory = chang14_ebook_first_model.integrate(time, 
                                                                    initial_condition=initial_condition, 
                                                                    input=chang14_ebook_first_light)
chang14_ebook_first_H_plus, chang14_ebook_first_H_minus = H_thresholds(chang14_ebook_first_model, chang14_ebook_first_trajectory)
# Chang14 Ebook second simulation
chang14_ebook_second_model = Skeldon23(params=parameters)
chang14_ebook_second_trajectory = chang14_ebook_second_model.integrate(time, 
                                                    initial_condition=initial_condition,
                                                    input=chang14_ebook_second_light)
chang14_ebook_second_H_plus, chang14_ebook_second_H_minus = H_thresholds(chang14_ebook_second_model, chang14_ebook_second_trajectory)
# Chang14 High Ebook first simulation
chang14_high_ebook_first_model = Skeldon23(params=parameters)
chang14_high_ebook_first_trajectory = chang14_high_ebook_first_model.integrate(time,
                                                                            initial_condition=initial_condition,
                                                                            input=chang14_high_ebook_first_light)
chang14_high_ebook_first_H_plus, chang14_high_ebook_first_H_minus = H_thresholds(chang14_high_ebook_first_model, chang14_high_ebook_first_trajectory)
# Chang14 High Ebook second simulation
chang14_high_ebook_second_model = Skeldon23(params=parameters)
chang14_high_ebook_second_trajectory = chang14_high_ebook_second_model.integrate(time,
                                                                            initial_condition=initial_condition,
                                                                            input=chang14_high_ebook_second_light)
chang14_high_ebook_second_H_plus, chang14_high_ebook_second_H_minus = H_thresholds(chang14_high_ebook_second_model, chang14_high_ebook_second_trajectory)
# Realistic Ebook first simulation
realistic_ebook_first_model = Skeldon23(params=parameters)
realistic_ebook_first_trajectory = realistic_ebook_first_model.integrate(time, 
                                                                        initial_condition=initial_condition, 
                                                                        input=realistic_ebook_first_light)
realistic_ebook_first_H_plus, realistic_ebook_first_H_minus = H_thresholds(realistic_ebook_first_model, realistic_ebook_first_trajectory)
# Realistic Ebook second simulation
realistic_ebook_second_model = Skeldon23(params=parameters)
realistic_ebook_second_trajectory = realistic_ebook_second_model.integrate(time, 
                                                                        initial_condition=initial_condition,
                                                                        input=realistic_ebook_second_light)
realistic_ebook_second_H_plus, realistic_ebook_second_H_minus = H_thresholds(realistic_ebook_second_model, realistic_ebook_second_trajectory)
# Save data
np.savez(f"{save_path}/chang14_simulation_reference_parameters.npz",
        time=time,
        parameters=parameters,
        initial_condition=initial_condition,
        chang14_ebook_first_trajectory_states=chang14_ebook_first_trajectory.states,
        chang14_ebook_first_light=chang14_ebook_first_light,
        chang14_ebook_first_sleep=chang14_ebook_first_model.sleep_state,
        chang14_ebook_first_H_plus=chang14_ebook_first_H_plus,
        chang14_ebook_first_H_minus=chang14_ebook_first_H_minus,
        chang14_ebook_second_trajectory_states=chang14_ebook_second_trajectory.states,
        chang14_ebook_second_light=chang14_ebook_second_light,
        chang14_ebook_second_sleep=chang14_ebook_second_model.sleep_state,
        chang14_ebook_second_H_plus=chang14_ebook_second_H_plus,
        chang14_ebook_second_H_minus=chang14_ebook_second_H_minus,
        chang14_high_ebook_first_trajectory_states=chang14_high_ebook_first_trajectory.states,
        chang14_high_ebook_first_light=chang14_high_ebook_first_light,
        chang14_high_ebook_first_sleep=chang14_high_ebook_first_model.sleep_state,
        chang14_high_ebook_first_H_plus=chang14_high_ebook_first_H_plus,
        chang14_high_ebook_first_H_minus=chang14_high_ebook_first_H_minus,
        chang14_high_ebook_second_trajectory_states=chang14_high_ebook_second_trajectory.states,
        chang14_high_ebook_second_light=chang14_high_ebook_second_light,
        chang14_high_ebook_second_sleep=chang14_high_ebook_second_model.sleep_state,
        chang14_high_ebook_second_H_plus=chang14_high_ebook_second_H_plus,
        chang14_high_ebook_second_H_minus=chang14_high_ebook_second_H_minus,
        realistic_ebook_first_trajectory_states=realistic_ebook_first_trajectory.states,
        realistic_ebook_first_light=realistic_ebook_first_light,
        realistic_ebook_first_sleep=realistic_ebook_first_model.sleep_state,
        realistic_ebook_first_H_plus=realistic_ebook_first_H_plus,
        realistic_ebook_first_H_minus=realistic_ebook_first_H_minus,
        realistic_ebook_second_trajectory_states=realistic_ebook_second_trajectory.states,
        realistic_ebook_second_light=realistic_ebook_second_light,
        realistic_ebook_second_sleep=realistic_ebook_second_model.sleep_state,
        realistic_ebook_second_H_plus=realistic_ebook_second_H_plus,
        realistic_ebook_second_H_minus=realistic_ebook_second_H_minus,
)

# Without forced wake up

In [9]:
# Equilibrate the model
dt = 0.005 # hours
equilibration_days = 3
equilibration_loops = 10
equilibration_time = np.arange(0.0, 24 * equilibration_days, dt)
lights_on = 6.0
lights_off = 22.0
typical_indoor_lux = 1000.0
initial_sleep = 1.0 # asleep
forced_wakeup_threshold = 0.5 # lux
regular_schedule = LightSchedule.Regular(typical_indoor_lux, lights_on=lights_on,
                                         lights_off=lights_off)
equilibration_light = regular_schedule(equilibration_time)
# Equilibrate
parameters = {'mu': 19.0,
              'chi': 11.0,
              'Delta': 6.0,
              'p': 0.6,
              'tauc': 24.2,
              'k': 0.55,
              }
equilibration_model = Skeldon23(params=parameters)
initial_condition = equilibration_model.equilibrate(equilibration_time, 
                                                    equilibration_light,
                                                    num_loops=equilibration_loops,)


In [10]:
time = np.load(f"{light_data_location}/time.npy")

chang14_ebook_first_light = np.load(f"{light_data_location}/chang14_ebook_first_light.npy")
chang14_ebook_second_light = np.load(f"{light_data_location}/chang14_ebook_second_light.npy")
chang14_high_ebook_first_light = np.load(f"{light_data_location}/chang14_high_ebook_first_light.npy")
chang14_high_ebook_second_light = np.load(f"{light_data_location}/chang14_high_ebook_second_light.npy")
realistic_ebook_first_light = np.load(f"{light_data_location}/realistic_ebook_first_light.npy")
realistic_ebook_second_light = np.load(f"{light_data_location}/realistic_ebook_second_light.npy")

parameters = {'mu': 19.0,
              'chi': 11.0,
              'Delta': 6.0,
              'p': 0.6,
              'tauc': 24.2,
              'k': 0.55,
              }

# Chang14 Ebook first simulation
chang14_ebook_first_model = Skeldon23(params=parameters)
chang14_ebook_first_trajectory = chang14_ebook_first_model.integrate(time, 
                                                                    initial_condition=initial_condition, 
                                                                    input=chang14_ebook_first_light)
chang14_ebook_first_H_plus, chang14_ebook_first_H_minus = H_thresholds(chang14_ebook_first_model, chang14_ebook_first_trajectory)
# Chang14 Ebook second simulation
chang14_ebook_second_model = Skeldon23(params=parameters)
chang14_ebook_second_trajectory = chang14_ebook_second_model.integrate(time, 
                                                    initial_condition=initial_condition,
                                                    input=chang14_ebook_second_light)
chang14_ebook_second_H_plus, chang14_ebook_second_H_minus = H_thresholds(chang14_ebook_second_model, chang14_ebook_second_trajectory)
# Chang14 High Ebook first simulation
chang14_high_ebook_first_model = Skeldon23(params=parameters)
chang14_high_ebook_first_trajectory = chang14_high_ebook_first_model.integrate(time,
                                                                            initial_condition=initial_condition,
                                                                            input=chang14_high_ebook_first_light)
chang14_high_ebook_first_H_plus, chang14_high_ebook_first_H_minus = H_thresholds(chang14_high_ebook_first_model, chang14_high_ebook_first_trajectory)
# Chang14 High Ebook second simulation
chang14_high_ebook_second_model = Skeldon23(params=parameters)
chang14_high_ebook_second_trajectory = chang14_high_ebook_second_model.integrate(time,
                                                                            initial_condition=initial_condition,
                                                                            input=chang14_high_ebook_second_light)
chang14_high_ebook_second_H_plus, chang14_high_ebook_second_H_minus = H_thresholds(chang14_high_ebook_second_model, chang14_high_ebook_second_trajectory)
# Realistic Ebook first simulation
realistic_ebook_first_model = Skeldon23(params=parameters)
realistic_ebook_first_trajectory = realistic_ebook_first_model.integrate(time, 
                                                                        initial_condition=initial_condition, 
                                                                        input=realistic_ebook_first_light)
realistic_ebook_first_H_plus, realistic_ebook_first_H_minus = H_thresholds(realistic_ebook_first_model, realistic_ebook_first_trajectory)
# Realistic Ebook second simulation
realistic_ebook_second_model = Skeldon23(params=parameters)
realistic_ebook_second_trajectory = realistic_ebook_second_model.integrate(time, 
                                                                        initial_condition=initial_condition,
                                                                        input=realistic_ebook_second_light)
realistic_ebook_second_H_plus, realistic_ebook_second_H_minus = H_thresholds(realistic_ebook_second_model, realistic_ebook_second_trajectory)
# Save data
np.savez(f"{save_path}/chang14_simulation_reference_parameters_without_forced_wakeup.npz",
        time=time,
        parameters=parameters,
        initial_condition=initial_condition,
        chang14_ebook_first_trajectory_states=chang14_ebook_first_trajectory.states,
        chang14_ebook_first_light=chang14_ebook_first_light,
        chang14_ebook_first_sleep=chang14_ebook_first_model.sleep_state,
        chang14_ebook_first_H_plus=chang14_ebook_first_H_plus,
        chang14_ebook_first_H_minus=chang14_ebook_first_H_minus,
        chang14_ebook_second_trajectory_states=chang14_ebook_second_trajectory.states,
        chang14_ebook_second_light=chang14_ebook_second_light,
        chang14_ebook_second_sleep=chang14_ebook_second_model.sleep_state,
        chang14_ebook_second_H_plus=chang14_ebook_second_H_plus,
        chang14_ebook_second_H_minus=chang14_ebook_second_H_minus,
        chang14_high_ebook_first_trajectory_states=chang14_high_ebook_first_trajectory.states,
        chang14_high_ebook_first_light=chang14_high_ebook_first_light,
        chang14_high_ebook_first_sleep=chang14_high_ebook_first_model.sleep_state,
        chang14_high_ebook_first_H_plus=chang14_high_ebook_first_H_plus,
        chang14_high_ebook_first_H_minus=chang14_high_ebook_first_H_minus,
        chang14_high_ebook_second_trajectory_states=chang14_high_ebook_second_trajectory.states,
        chang14_high_ebook_second_light=chang14_high_ebook_second_light,
        chang14_high_ebook_second_sleep=chang14_high_ebook_second_model.sleep_state,
        chang14_high_ebook_second_H_plus=chang14_high_ebook_second_H_plus,
        chang14_high_ebook_second_H_minus=chang14_high_ebook_second_H_minus,
        realistic_ebook_first_trajectory_states=realistic_ebook_first_trajectory.states,
        realistic_ebook_first_light=realistic_ebook_first_light,
        realistic_ebook_first_sleep=realistic_ebook_first_model.sleep_state,
        realistic_ebook_first_H_plus=realistic_ebook_first_H_plus,
        realistic_ebook_first_H_minus=realistic_ebook_first_H_minus,
        realistic_ebook_second_trajectory_states=realistic_ebook_second_trajectory.states,
        realistic_ebook_second_light=realistic_ebook_second_light,
        realistic_ebook_second_sleep=realistic_ebook_second_model.sleep_state,
        realistic_ebook_second_H_plus=realistic_ebook_second_H_plus,
        realistic_ebook_second_H_minus=realistic_ebook_second_H_minus,
)